#### Notebook for running React experiments

In [1]:
import sys, os
sys.path.append('..')
root  = '../root/'

In [2]:
import joblib
from util import summarize_react_trial, log_react_trial, save_agents
from agents import ReactReflectAgent, ReactAgent, ReflexionStrategy

#### Load the HotpotQA Sample

In [3]:
hotpot = joblib.load('../data/hotpot-qa-distractor-sample.joblib').reset_index(drop = True)

In [3]:
from datasets import load_dataset

dataset = load_dataset("nazimali/quran-question-answer-context")

hotpot = dataset['train'].to_pandas()

#### Define the Reflexion Strategy

In [4]:
print(ReflexionStrategy.__doc__)


    NONE: No reflection
    LAST_ATTEMPT: Use last reasoning trace in context 
    REFLEXION: Apply reflexion to the next reasoning trace 
    LAST_ATTEMPT_AND_REFLEXION: Use last reasoning trace in context and apply reflexion to the next reasoning trace 
    


In [5]:
strategy: ReflexionStrategy = ReflexionStrategy.REFLEXION

#### Initialize a React Agent for each question

In [6]:
agent_cls = ReactReflectAgent if strategy != ReflexionStrategy.NONE else ReactAgent
agents = [agent_cls(row['question'], row['answer']) for _, row in hotpot.iterrows()]

#### Run `n` trials

In [7]:
n = 5
trial = 0
log = ''

In [8]:
for i in range(n):
    for agent in [a for a in agents if not a.is_correct()]:
        if strategy != ReflexionStrategy.NONE:
            agent.run(reflect_strategy = strategy)
        else:
            agent.run()
        print(f'Answer: {agent.key}')
    trial += 1
    log += log_react_trial(agents, trial)
    correct, incorrect, halted = summarize_react_trial(agents)
    print(f'Finished Trial {trial}, Correct: {len(correct)}, Incorrect: {len(incorrect)}, Halted: {len(halted)}')

Thought 1: I need to search for the virtues of Allah upon his prophet Dawood (peace be upon him).
Action 1: Search[فضائل الله على نبيه داود]
Observation 1: Could not find [فضائل الله على نبيه داود]. Similar: []
Thought 2: I couldn't find an exact match for the search query. Let me try searching for the virtues of Prophet Dawood (peace be upon him) first.
Action 2: Search[فضائل النبي داود]
Observation 2: Could not find [فضائل النبي داود]. Similar: []


KeyboardInterrupt: 

#### Save the result log

In [ ]:
with open(os.path.join(root, 'ReAct', strategy.value, f'{len(agents)}_questions_{trial}_trials.txt'), 'w') as f:
    f.write(log)
save_agents(agents, os.path.join('ReAct', strategy.value, 'agents'))